In [1]:
function qr_real_pivot!(U::Matrix{ComplexF64}, input::Matrix{ComplexF64}, squared_norms::Vector{Float64}, pivot::Vector{Int64}, 
        n::Int64 = size(input,1))
    @inbounds begin
        @turbo for i = 1:n
            pivot[i] = i
        end
    
        # 对n个k维矢量进行排序 与 householder 交叉进行        
        # 找到第1个向量
        current_index = 1
        tmp_pos = 0

        max_norm2 = 0.0
        tmp_norm2 = 0.0
        find_max_column!(input, squared_norms, pivot, tmp_norm2, max_norm2, current_index, tmp_pos, n)
        squared_norms[1] = house_holder!(input, squared_norms[1], tmp_norm2, max_norm2, 1, n)
        # 找第2到n-1个向量
        for i in 2:n-1
            max_norm2 = 0.0
            current_index = i
            
            find_max_column!(input, squared_norms, pivot, tmp_norm2, max_norm2, current_index, tmp_pos, n, i)
            squared_norms[i] = house_holder!(input, squared_norms[i], tmp_norm2, max_norm2, i, n)
        end

        # 接下来开始寻找幺正矩阵U
        copyto!(U, I)

        tmp_norm2 = input[n,n]
        input[n,n] = sign(tmp_norm2)
        squared_norms[n] = abs(tmp_norm2)
        
        for k = (n-1):-1:1
            tmp_norm2 = squared_norms[k]
            get_unitary!(input, U, max_norm2, tmp_norm2, k, n)
            
            # TODO::也许可以把这个去掉,只要我们心中知道它是零即可
            # @turbo for i in (k+1):n
            #     input[i,k] = 0.0
            # end

            tmp_norm2 = input[k,k]
            input[k,k] = sign(tmp_norm2)
            tmp_norm2 = abs(tmp_norm2)
            squared_norms[k] = tmp_norm2
            tmp_norm2 = 1 / tmp_norm2

            @turbo for j in k+1:n
                input[k,j] *= tmp_norm2
            end
            
        end
        nothing
    end
end



function find_max_column!(input::Matrix{ComplexF64}, squared_norms::Vector{Float64}, pivot::Vector{Int64}, tmp_norm2::Float64, max_norm2::Float64, current_index::Int, tmp_pos::Int, n::Int) # true 代表就是对第一个向量
    for i in 1:n
        @turbo for j = 1:n
            tmp_norm2 += abs2(input[j,i])
        end
        
        squared_norms[i] = tmp_norm2
        if tmp_norm2 > max_norm2
            max_norm2 = tmp_norm2
            current_index = i
        end
        tmp_norm2 = 0.0
    end
    
    if current_index != 1
        pivot[1] = pivot[current_index]
        pivot[current_index] = 1
        
        ## 下面这样会比较慢, 建议用@turbo来进行加速
        # vector_tmp = input[:, current_index]
        # input[:, current_index] = input[:, 1]
        # input[:, 1] = vector_tmp
        @turbo for j in 1:n
            tmp_norm2 = input[j, current_index]
            input[j, current_index] = input[j, 1]
            input[j, 1] = tmp_norm2
        end
        squared_norms[current_index] = squared_norms[1]
    end
    squared_norms[1] = sqrt(max_norm2)
    nothing
end


@inline function get_unitary!(input::Matrix{Float64}, U_test::Matrix{Float64}, max_norm2::Float64, tmp_norm2::Float64, k::Int, n::Int)
    # tmp_norm2::存放β
    ## 首先计算U[k:n,k]
    ## 然后计算U[k, (k+1):n]
    ## 这两个for-loop放在一起
    U_test[k,k] -= tmp_norm2
    @turbo for i in (k+1):n ## TODO::修改一下@turbo看看有没有加速
        max_norm2 = -(input[i,k] * tmp_norm2)
        U_test[i,k] += max_norm2
        for j in (k+1):n
            U_test[k,j] += max_norm2 * U_test[i, j]
        end
    end
    
    # @turbo for i in (k+1):n
    #     tmp_norm2 = input[i,k]
    #     input[i,k] -= tmp_norm2
    #     for j in (k+1):n
    #         U_test[i,j] += tmp_norm2 * U_test[k,j]  # 是用+号而不是用-号
    #     end
    # end

    @turbo for i in (k+1):n ## TODO::修改一下@turbo看看有没有加速  22/116
        for j in (k+1):n
            U_test[i,j] += input[i,k] * U_test[k,j]  # 是用+号而不是用-号  ## TODO::修改一下这个loop
        end
    end

    ## 注意: 下面这种@turbo算出来的结果是错误的，可能跟@turbo的用法有关
    # @turbo for i in (k+1):n
    #     for j in (k+1):n
    #         U_test[i,j] += input[i,k] * U_test[k,j]  # 是用+号而不是用-号
    #     end
    #     input[i,k] = 0.0
    # end

end

LoadError: LoadError: UndefVarError: @turbo not defined
in expression starting at In[1]:4

In [6]:
(1 + 2im)*(2 - 3im)
real(1 + 2im)
imag(1 + 2im)
conj(1 + 2im)
abs(1 + 2im)
abs2(1 + 2im)
angle(1 + 2im)
a = 1; b = 2; complex(a, b)

1 + 2im

In [8]:
A = rand(ComplexF64, 10,10)

10×10 Matrix{ComplexF64}:
  0.309886+0.163234im   0.387371+0.00968791im  …  0.453449+0.303233im
  0.775472+0.017508im    0.65161+0.568593im       0.751475+0.155021im
  0.645202+0.0965552im  0.609173+0.1365im         0.162038+0.220783im
  0.635724+0.156963im   0.570216+0.239083im       0.720395+0.461146im
  0.156169+0.612721im    0.87254+0.969302im       0.383749+0.44182im
  0.577467+0.845548im   0.275585+0.876439im    …  0.818506+0.0050653im
  0.954373+0.997168im   0.672916+0.475954im       0.703302+0.817211im
 0.0691908+0.957491im   0.876375+0.218943im       0.816007+0.901338im
  0.293601+0.0282106im  0.734233+0.279864im       0.517847+0.449874im
   0.20586+0.804822im   0.630779+0.99723im        0.179158+0.0984252im

In [9]:
A = zeros(ComplexF64, 10,10)

10×10 Matrix{ComplexF64}:
 0.0+0.0im  0.0+0.0im  0.0+0.0im  …  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  …  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im

In [10]:
A = ones(ComplexF64, 10,10)

10×10 Matrix{ComplexF64}:
 1.0+0.0im  1.0+0.0im  1.0+0.0im  …  1.0+0.0im  1.0+0.0im  1.0+0.0im
 1.0+0.0im  1.0+0.0im  1.0+0.0im     1.0+0.0im  1.0+0.0im  1.0+0.0im
 1.0+0.0im  1.0+0.0im  1.0+0.0im     1.0+0.0im  1.0+0.0im  1.0+0.0im
 1.0+0.0im  1.0+0.0im  1.0+0.0im     1.0+0.0im  1.0+0.0im  1.0+0.0im
 1.0+0.0im  1.0+0.0im  1.0+0.0im     1.0+0.0im  1.0+0.0im  1.0+0.0im
 1.0+0.0im  1.0+0.0im  1.0+0.0im  …  1.0+0.0im  1.0+0.0im  1.0+0.0im
 1.0+0.0im  1.0+0.0im  1.0+0.0im     1.0+0.0im  1.0+0.0im  1.0+0.0im
 1.0+0.0im  1.0+0.0im  1.0+0.0im     1.0+0.0im  1.0+0.0im  1.0+0.0im
 1.0+0.0im  1.0+0.0im  1.0+0.0im     1.0+0.0im  1.0+0.0im  1.0+0.0im
 1.0+0.0im  1.0+0.0im  1.0+0.0im     1.0+0.0im  1.0+0.0im  1.0+0.0im